#### Importação de bibliotecas

In [109]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

#### Importação das base de dados

In [110]:
dftrain = pd.read_csv('train.csv')
dftest = pd.read_csv('test.csv')

#### Verificação das correlações com a variável alvo (variável alvo: NU_NOTA_MT - Nota de matemática)

In [129]:
math_correlation = pd.DataFrame(dftrain.corr()['NU_NOTA_MT'].sort_values(ascending = False))

In [130]:
math_correlation.head(10)

,NU_NOTA_MT
NU_NOTA_MT,1.000000
NU_NOTA_CN,0.584941
NU_NOTA_CH,0.529594
NU_NOTA_LC,0.494695
NU_NOTA_REDACAO,0.379376
TP_DEPENDENCIA_ADM_ESC,0.373233
NU_NOTA_COMP3,0.350307
NU_NOTA_COMP5,0.343337
NU_NOTA_COMP4,0.342282
NU_NOTA_COMP2,0.335638


In [131]:
math_correlation = pd.DataFrame(dftrain.corr()['NU_NOTA_MT'].sort_values(ascending = True))

In [132]:
math_correlation.head(10)

,NU_NOTA_MT
Q037,-0.265249
Q038,-0.202259
TP_LINGUA,-0.201726
TP_COR_RACA,-0.157364
Q036,-0.145501
Q041,-0.131795
Q030,-0.127963
Q029,-0.124075
Q040,-0.119997
Q033,-0.114083


##### Por comodidade, serão utilizadas as quatro primeiras variáveis com correlação positiva mais alta, já com tratamento dos dados faltantes:

In [115]:
x = dftrain[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO']].fillna(-1000)
y = dftrain[['NU_NOTA_MT']].fillna(-1000)

#### Separando os grupos de treino e de teste

##### Foi utilizados um grupo de treino bem maior em relação ao de teste para se ter um modelo mais bem treino, afinal, este modelo só será utilizados na base de dados de teste (dftest)

In [117]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.99)

#### Aplicação do modelo de Random Forest

In [118]:
random_forest = RandomForestRegressor(n_estimators = 100)
random_forest.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

#### Atribuindo à variável Xres as variáveis da base de dados de teste

In [119]:
Xres = dftest[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO']].fillna(-1000)

#### Fazendo o "fit" dos dados

In [120]:
random_forest = RandomForestRegressor(n_estimators = 100)
random_forest.fit(x, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

#### Predizendo os valores da nota de matemática com base na variável Xres que possui todas as variáveis usadas para predição

In [121]:
Yres = random_forest.predict(Xres)

##### A predição atribui valores negativos a certas notas com as variáveis Xres, isso se deve, por exemplo, às notas que são zeradas pelos alunos e o modelo prediz a nota de matemática como negativa. Para isso, foram atribuídas notas também zeradas para matemática, uma vez que não existe nota negativa para este caso

In [122]:
Yres[Yres < 0] = 0

#### Arredondando valores

In [123]:
dftest['NU_NOTA_MT'] = np.around(Yres, 2)

#### Atribuindos as notas de matemática à base de dados de teste

In [124]:
dftest = dftest[['NU_INSCRICAO','NU_NOTA_MT']]

#### Transformando a base de dados em um arquivo .csv

In [125]:
dftest.to_csv('answer.csv', index=False)